# **Import necessary libraries**

In [ ]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import random
import copy

# **Define variables for optimizer function and data processing**

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define variables for optimizer function and data processing
NUMOFCLIENTS = 10
SELECT_CLIENTS = 0.5
EPOCHS = 5
CLIENT_EPOCHS = 5
BATCH_SIZE = 10
DROP_RATE = 0

# Model config
LOSS = nn.CrossEntropyLoss()
NUMOFCLASSES = 10
lr = 0.0025
OPTIMIZER = optim.SGD  # You can customize the optimizer later if needed

## Writes the data into a csv file

### Part of preprocessing

In [ ]:
def write_csv(method_name, list):
    file_name = '{name}_CIFAR10_randomDrop_{drop}%_output_C_{c}_LR_{lr}_CLI_{cli}_CLI_EPOCHS_{cli_epoch}_TOTAL_EPOCHS_{epochs}_BATCH_{batch}.csv'
    file_name = file_name.format(folder="origin_drop",drop=DROP_RATE, name=method_name, c=SELECT_CLIENTS, lr=lr, cli=NUMOFCLIENTS, cli_epoch=CLIENT_EPOCHS, epochs=EPOCHS, batch=BATCH_SIZE)
    f = open(file_name, 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)

    for l in list:
        wr.writerow(l)
    f.close()

## **Continue data preprocessing**

Prepare training and testing sets

In [ ]:
def load_dataset():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return train_loader, test_loader

## **Instantiate Model**

In [ ]:
class FlModel(nn.Module):
    def __init__(self):
        super(FlModel, self).__init__()
        # Define the model architecture based on the one you built in the previous section
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout1 = nn.Dropout2d(0.2)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout2 = nn.Dropout2d(0.2)

        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout3 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, NUMOFCLASSES)

    def forward(self, x):
        # Define the forward pass here
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)

        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)

        return F.softmax(x, dim=1)


## **Splits the training and testing sets among the available clients**

Decentralized learning

In [ ]:
def client_data_config(train_loader):
    client_data = [() for _ in range(NUMOFCLIENTS)]
    num_of_each_dataset = len(train_loader.dataset) // NUMOFCLIENTS

    for i in range(NUMOFCLIENTS):
        split_data_index = random.sample(range(len(train_loader.dataset)), num_of_each_dataset)
        new_x_train = torch.stack([train_loader.dataset[k][0] for k in split_data_index])
        new_y_train = torch.tensor([train_loader.dataset[k][1] for k in split_data_index])

        client_data[i] = (new_x_train, new_y_train)

    return client_data

## **Implementation of Federated Average Algorithm**

Computation of average weights of a model across clients

In [ ]:
def fedAVG(server_state_dict, client_state_dicts):
    avg_state_dict = copy.deepcopy(server_state_dict)

    if len(client_state_dicts) > 0:
        for key in avg_state_dict.keys():
            for client_state_dict in client_state_dicts:
                avg_state_dict[key] += client_state_dict[key]

            avg_state_dict[key] /= len(client_state_dicts)

    return avg_state_dict

Accuracy Function

In [ ]:
def calculate_accuracy(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Get predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Update counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy


## **Updates the client models by running them through the existent models assigned to the clients**

In [ ]:
def client_update(index, client, now_epoch, avg_state_dict, train_loader):
    print(f"client {index + 1}/{len(selected_model)} fitting")

    if now_epoch != 0:
        client.load_state_dict(avg_state_dict)

    # Define the optimizer and other necessary settings
    optimizer = OPTIMIZER(client.parameters(), lr=lr)

    for epoch in range(CLIENT_EPOCHS):
        # Your training loop here
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = client(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Print loss for each iteration
            # print(f"Client {index + 1}/{len(selected_model)} - Epoch {now_epoch + 1}/{EPOCHS} - Loss: {loss.item()}")

            accuracy = calculate_accuracy(client, train_loader)
            client.accuracy = accuracy  # Store accuracy as an attribute
            print(f"Client {index + 1}/{int(NUMOFCLIENTS * SELECT_CLIENTS)} - Epoch {epoch + 1}/{CLIENT_EPOCHS} - Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

    return client.state_dict()

## **Runs federated learning code on client datasets, displaying epochs, weighted average, loss, and accuracy.**

In [ ]:
if __name__ == "__main__":
    # Load dataset and create DataLoader
    train_loader, test_loader = load_dataset()

    # Instantiate the server model and move it to the appropriate device (GPU if available)
    server_model = FlModel().to(device)

    # Generate client data configurations
    client_data = client_data_config(train_loader)

    # Instantiate client models and move them to the appropriate device
    fl_model = [FlModel().to(device) for _ in range(NUMOFCLIENTS)]

    # Initialize the state dictionary for FedAvg
    avg_state_dict = copy.deepcopy(server_model.state_dict())

    # List to store evaluation accuracy for each epoch
    server_evaluate_acc = []

    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(EPOCHS):
        client_state_dicts = []

        # Randomly select a subset of clients for this round
        selected_num = int(max(NUMOFCLIENTS * SELECT_CLIENTS, 1))
        split_data_index = random.sample(range(NUMOFCLIENTS), selected_num)
        selected_model = [fl_model[k] for k in split_data_index]

        # Perform client updates and gather client states
        for index, client in enumerate(selected_model):
            client_state_dict = client_update(index, client, epoch, avg_state_dict, train_loader)
            client_state_dicts.append(client_state_dict)

        # Aggregate client states using FedAVG
        avg_state_dict = fedAVG(avg_state_dict, client_state_dicts)

        # Update the server model with the aggregated state
        server_model.load_state_dict(avg_state_dict)

    # Evaluate the server model on the test set
    server_model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = server_model(inputs)

            # Get predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Update counts
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and print accuracy
    accuracy = correct / total
    print(f"Epoch {epoch + 1}/{EPOCHS} - Accuracy on test set: {accuracy}")

    # Save accuracy for logging or further analysis
    server_evaluate_acc.append(accuracy)

    # Write the evaluation results to a CSV file
    write_csv("FedAvg", server_evaluate_acc)

100%|██████████| 170498071/170498071 [00:02<00:00, 60275243.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
client 1/5 fitting
Client 1/5 - Epoch 1/5 - Loss: 2.3018, Accuracy: 0.1170
Client 1/5 - Epoch 1/5 - Loss: 2.3015, Accuracy: 0.1167
Client 1/5 - Epoch 1/5 - Loss: 2.3035, Accuracy: 0.1170
Client 1/5 - Epoch 1/5 - Loss: 2.3023, Accuracy: 0.1171
Client 1/5 - Epoch 1/5 - Loss: 2.3017, Accuracy: 0.1172
Client 1/5 - Epoch 1/5 - Loss: 2.3023, Accuracy: 0.1170
Client 1/5 - Epoch 1/5 - Loss: 2.3021, Accuracy: 0.1173
Client 1/5 - Epoch 1/5 - Loss: 2.3029, Accuracy: 0.1172
Client 1/5 - Epoch 1/5 - Loss: 2.3021, Accuracy: 0.1169
Client 1/5 - Epoch 1/5 - Loss: 2.3021, Accuracy: 0.1167
Client 1/5 - Epoch 1/5 - Loss: 2.3036, Accuracy: 0.1168
Client 1/5 - Epoch 1/5 - Loss: 2.3041, Accuracy: 0.1169
Client 1/5 - Epoch 1/5 - Loss: 2.3026, Accuracy: 0.1171
Client 1/5 - Epoch 1/5 - Loss: 2.3017, Accuracy: 0.1170
Client 1/5 - Epoch 1/5 - Loss: 2.3018, Accuracy: 0.1170
Client 1/5 - Epoch 1/5 - Loss: 2.3021, Accuracy: 0.1